# 응용통계학 과제
> 다중공선성

- toc:true
- branch: master
- badges: true
- comments: false
- author: 최서연
- categories: [Applied Statistics, 다중공선성]

# 응용통계학 과제 20220512
- 202150754 최서연
> 다중공선성 


다중공선성이 존재하는 상황을 가정하고 

 

다중공선성을 어느 정도 제거한 모형 (M1)과 다중공선성이 내재되어 있는 모형 (M2) 을 고려하여

 

두 모형의 예측력을 모의실험을 통해 비교하여라,

 

단, 실험은 여러 번 반복하여 평균적인 결과를  report하되 설명변수의 개수는 3개 이상으로 설정하여라.

 

이미 존재하는 문서들을 참고하거나 재현해도 무방함.

 

(첨부된 문서 참고)

In [1]:
Nrep = 200

n = 1000

hatb1 <- hatb2 <- c()
te1 <- te2 <- c()

for (k in 1:Nrep)
{
x1 = runif(n,-3,3)
x2 = x1 + rnorm(n,0,0.01)
#X = model.matrix(~ x1 + x2)
#solve(t(X)%*%X)
y = 2 + x1 + 2*x2 + rnorm(n)

ind = sample(1:n,500)
tx1 = x1[ind]
tx2 = x2[ind]
test_x1 = x1[-ind]
test_x2 = x2[-ind]
ty  = y[ind]
test_y = y[-ind]

fit1 = lm(ty~tx1+tx2)
fit2 = lm(ty~tx1)

hatb1[k] = fit1$coefficients[2]
hatb2[k] = fit2$coefficients[2]

te1[k] = mean((test_y - predict(fit1,newdata=data.frame(test_x1,test_x2)))^2)
te2[k] = mean((test_y - predict(fit2,newdata=data.frame(test_x1)))^2)
#summary(lm(y~x1+x2))
#print(k)
}

c(mean(te1),mean(te2))
c(mean((hatb1-1)^2),mean((hatb2-2)^2))

[1] 55.60855 55.60716

[1] 22.1862442  0.9971952

---

In [28]:
library(regclass)

In [29]:
library(car)

---

## 시도 1

In [4]:
Nrep = 200

n = 1000

hatb1 <- hatb2 <- c()
te1 <- te2 <- c()

for (k in 1:Nrep)
{
x1 = runif(n,-3,3)
x2 = x1 + rnorm(n,0,0.01)
x3 = x1^2 + rnorm(n,0,0.01)
y = 2 + x1 + 2*x2 + 3*x3 + rnorm(n)

ind = sample(1:n,500)
tx1 = x1[ind]
tx2 = x2[ind]
tx3 = x3[ind]
test_x1 = x1[-ind]
test_x2 = x2[-ind]
test_x3 = x3[-ind]
ty  = y[ind]
test_y = y[-ind]

M2 = lm(ty~tx1+tx2+tx3)
M1 = lm(ty~tx1)

hatb1[k] = M2$coefficients[2]
hatb2[k] = M1$coefficients[2]

te1[k] = mean((test_y - predict(M1,newdata=data.frame(test_x1,test_x2,test_x3)))^2)
te2[k] = mean((test_y - predict(M1,newdata=data.frame(test_x1)))^2)
}

c(mean(te1),mean(te2))
c(mean((hatb1-2)^2),mean((hatb2-1)^2))

[1] 119.953 119.953

[1] 22.765494  4.044129

---

## 시도 2

- $y_1=3+x_1+1.5x_2+3.5x_3+\epsilon$
  - $x_2=x_1+\epsilon$
  - $x_3=x_1+\epsilon$
- 1000번 반복
- 500개 데이터

In [5]:
x1 = runif(500)
x2 = x1 + rnorm(500,0,0.01)
x3 = x1 + rnorm(500,0,0.1)
y1= 3 + x1 + 1.5*x2 + 3.5*x3 + rnorm(500)

In [6]:
cor(x1,x2)
cor(x1,x3)

[1] 0.9994442

[1] 0.9415443

높은 상관계수 확인

In [7]:
M1 = lm(y1~x1)

In [8]:
M2 = lm(y1~x1+x2+x3)

In [9]:
VIF(M2)

x1         x2         x3 
903.701250 900.495402   8.817672

다중공선성 존재를 가정한 모형의 VIF 10 넘는 모습이었다.

In [10]:
print(M1$coefficients)

(Intercept)          x1 
   3.059741    5.870505 


In [11]:
print(M2$coefficients)

(Intercept)          x1          x2          x3 
  3.0182881   2.0232424   0.2413279   3.7029953 


In [12]:
mean((y1-predict(M1,data.frame(x1)))^2)

[1] 1.171279

In [13]:
mean((y1-predict(M2,data.frame(x1,x2,x3)))^2)

[1] 1.033291

다중공선성을 제거한 모형이 다중공선성이 있는 모형보다 제곱평균오차가 컸다.

반복

In [14]:
result1 = c()
result2 = c()
for (i in 1:1000){
    x1 = runif(500)
    x2 = x1 + rnorm(500,0,0.01)
    x3 = x1 + rnorm(500,0,0.01)
    y1= 3 + x1 + 1.5*x2 + 3.5*x3 + rnorm(500)
    train_x1 = x1[1:250]
    train_x2 = x2[1:250]
    train_x3 = x3[1:250]
    train_y1 = y1[1:250]
    test_x1 = x1[251:500]
    test_x2 = x2[251:500]
    test_x3 = x3[251:500]
    test_y1 = y1[251:500]
    M1 = lm(train_y1~train_x1)
    M2 = lm(train_y1~train_x1+train_x2+train_x3)
    result1[i]=mean((test_y1-predict(M1,data.frame(test_x1)))^2)
    result2[i]=mean((test_y1-predict(M2,data.frame(test_x1,test_x2,test_x3)))^2)
    }
print(mean(result1));print(mean(result2))

[1] 7.024189
[1] 7.033023


다중공선성이 있는 모형과 다중공선성이 없는 모형의 MSE가 비슷한 값이 나왔다.

In [15]:
M2$coefficients

(Intercept)    train_x1    train_x2    train_x3 
  3.0054319  11.7483379  -6.4915223   0.8064938

---

## 시도 3

- $y_1=3+x_1+2x_2+3x_3+\epsilon$
  - $x_2=x_1^2+\epsilon$
  - $x_3=x_1^3+\epsilon$
- 1000번 반복
- 500개 데이터

In [16]:
result1 = c()
result2 = c()
for (i in 1:1000){
    x1 = runif(1000)
    x2 = x1^2 + rnorm(1000,0,0.01)
    x3 = x1^3 + rnorm(1000,0,0.01)
    y1= 3 + x1 + 2*x2 + 3*x3 + rnorm(1000)
    train_x1 = x1[1:500]
    train_x2 = x2[1:500]
    train_x3 = x3[1:500]
    train_y1 = y1[1:500]
    test_x1 = x1[501:1000]
    test_x2 = x2[501:1000]
    test_x3 = x3[501:1000]
    test_y1 = y1[501:1000]
    M1 = lm(train_y1~train_x1)
    M2 = lm(train_y1~train_x1+train_x2+train_x3)
    result1[i]=mean((test_y1-predict(M1,data.frame(test_x1)))^2)
    result2[i]=mean((test_y1-predict(M2,data.frame(test_x1,test_x2,test_x3)))^2)
    }
print(mean(result1));print(mean(result2))

[1] 6.652483
[1] 6.896817


In [17]:
M2$coefficients

(Intercept)    train_x1    train_x2    train_x3 
   2.715989    3.657056   -4.074019    7.062712

---

## 시도4

- $y_1=3+x_1+2x_2+3x_3+\epsilon$
  - $x_2=x_1^2+\epsilon$
  - $x_3=x_1^3+\epsilon$
- 1000번 반복
- 1000개 데이터

In [18]:
result1 = c()
result2 = c()
for (i in 1:1000){
    x1 = runif(1000)
    x2 = x1^2 + rnorm(1000,0,0.01)
    x3 = x1^3 + rnorm(1000,0,0.01)
    y1= 3 + x1 + 2*x2 + 3*x3 + rnorm(1000,0,0.1)
    train_x1 = x1[1:500]
    train_x2 = x2[1:500]
    train_x3 = x3[1:500]
    train_y1 = y1[1:500]
    test_x1 = x1[501:1000]
    test_x2 = x2[501:1000]
    test_x3 = x3[501:1000]
    test_y1 = y1[501:1000]
    M1 = lm(train_y1~train_x1)
    M2 = lm(train_y1~train_x1+train_x2+train_x3)
    result1[i]=mean((test_y1-predict(M1,data.frame(test_x1)))^2)
    result2[i]=mean((test_y1-predict(M2,data.frame(test_x1,test_x2,test_x3)))^2)
    }
print(mean(result1));print(mean(result2))

[1] 5.69406
[1] 5.929241


In [19]:
M2$coefficients

(Intercept)    train_x1    train_x2    train_x3 
   2.977275    1.164406    1.628985    3.258352

---

## 시도 5

- $y_1=3+x_1+5x_2+10x_3+\epsilon$
  - $x_2=2x_1+\epsilon$
  - $x_3=3x_1+\epsilon$
- 1000번 반복
- 1000개 데이터

In [20]:
result1 = c()
result2 = c()
for (i in 1:1000){
    x1 = runif(1000)
    x2 = x1*2 + rnorm(1000,0,0.01)
    x3 = x1*3 + rnorm(1000,0,0.01)
    y1= 3 + x1 + 5*x2 + 10*x3 + rnorm(1000)
    train_x1 = x1[1:500]
    train_x2 = x2[1:500]
    train_x3 = x3[1:500]
    train_y1 = y1[1:500]
    test_x1 = x1[501:1000]
    test_x2 = x2[501:1000]
    test_x3 = x3[501:1000]
    test_y1 = y1[501:1000]
    M1 = lm(train_y1~train_x1)
    M2 = lm(train_y1~train_x1+train_x2+train_x3)
    result1[i]=mean((test_y1-predict(M1,data.frame(test_x1)))^2)
    result2[i]=mean((test_y1-predict(M2,data.frame(test_x1,test_x2,test_x3)))^2)
    }
print(mean(result1));print(mean(result2))

[1] 280.6168
[1] 280.6332


In [21]:
M2$coefficients

(Intercept)    train_x1    train_x2    train_x3 
   2.978875   25.189961    3.118960    3.222922

- $x_1$이랑 제곱관계였던 시도 4까지의 결과와 조금 다르게 거의 비슷한 모습

---

## 시도 6

- $y_1=3+x_1+5x_2+10x_3+\epsilon$
  - $x_2=2x_1+\epsilon$
  - $x_3=3x_1+\epsilon$
  - $x_4=4x_1+\epsilon$
- 1000번 반복
- 1000개 데이터

In [22]:
result1 = c()
result2 = c()
for (i in 1:1000){
    x1 = runif(1000)
    x2 = x1*2 + rnorm(1000,0,0.01)
    x3 = x1*3 + rnorm(1000,0,0.01)
    x4 = x1*4 + rnorm(1000,0,0.01)
    y1= 3 + x1 + 5*x2 + 10*x3 + 15*x4 + rnorm(1000)
    train_x1 = x1[1:500]
    train_x2 = x2[1:500]
    train_x3 = x3[1:500]
    train_x4 = x4[1:500]
    train_y1 = y1[1:500]
    test_x1 = x1[501:1000]
    test_x2 = x2[501:1000]
    test_x3 = x3[501:1000]
    test_x4 = x4[501:1000]
    test_y1 = y1[501:1000]
    M1 = lm(train_y1~train_x1)
    M2 = lm(train_y1~train_x1+train_x2+train_x3++train_x4)
    result1[i]=mean((test_y1-predict(M1,data.frame(test_x1)))^2)
    result2[i]=mean((test_y1-predict(M2,data.frame(test_x1,test_x2,test_x3,test_x4)))^2)
    }
print(mean(result1));print(mean(result2))

[1] 1700.399
[1] 1700.447


In [23]:
M2$coefficients

(Intercept)    train_x1    train_x2    train_x3    train_x4 
   2.948311  -23.476413   19.486877   10.091052   13.783955

---

## 시도 7

- $y_1=3+x_1+5x_2+10x_3+15x_4+20x_5+\epsilon$
  - $x_2=2x_1+\epsilon$
  - $x_3=3x_1+\epsilon$
  - $x_4=4x_1+\epsilon$
  - $x_5=5x_1+\epsilon$
- 1000번 반복
- 1000개 데이터
- 설명변수를 하나씩 제거해나가며 값 확인

In [24]:
result1 = c()
result2 = c()
result2_2 = c()
result2_3 = c()
result2_4 = c()
for (i in 1:1000){
    x1 = runif(1000)
    x2 = x1*2 + rnorm(1000,0,0.01)
    x3 = x1*3 + rnorm(1000,0,0.01)
    x4 = x1*4 + rnorm(1000,0,0.01)
    x5 = x1*5 + rnorm(1000,0,0.01)
    y1= 3 + x1 + 5*x2 + 10*x3 + 15*x4 + 20*x5 + rnorm(1000)
    train_x1 = x1[1:500]
    train_x2 = x2[1:500]
    train_x3 = x3[1:500]
    train_x4 = x4[1:500]
    train_x5 = x5[1:500]
    train_y1 = y1[1:500]
    test_x1 = x1[501:1000]
    test_x2 = x2[501:1000]
    test_x3 = x3[501:1000]
    test_x4 = x4[501:1000]
    test_x5 = x5[501:1000]
    test_y1 = y1[501:1000]
    M1 = lm(train_y1~train_x1)
    M2 = lm(train_y1~train_x1+train_x2)
    M2_2 = lm(train_y1~train_x1+train_x2+train_x3)
    M2_3 = lm(train_y1~train_x1+train_x2+train_x3+train_x4)
    M2_4 = lm(train_y1~train_x1+train_x2+train_x3+train_x4+train_x5)
    result1[i]=mean((test_y1-predict(M1,data.frame(test_x1)))^2)
    result2[i]=mean((test_y1-predict(M2,data.frame(test_x1,test_x2,test_x3)))^2)
    result2_2[i]=mean((test_y1-predict(M2_2,data.frame(test_x1,test_x2,test_x3)))^2)
    result2_3[i]=mean((test_y1-predict(M2_3,data.frame(test_x1,test_x2,test_x3,test_x4)))^2)
    result2_4[i]=mean((test_y1-predict(M2_4,data.frame(test_x1,test_x2,test_x3,test_x4,test_x5)))^2)
    }
print(mean(result1));print(mean(result2));print(mean(result2_2));print(mean(result2_3));print(mean(result2_4))

[1] 6736.468
[1] 6736.455
[1] 6736.485
[1] 6736.49
[1] 6736.483


In [25]:
M2$coefficients;M2_2$coefficients;M2_3$coefficients;M2_4$coefficients

(Intercept)    train_x1    train_x2 
   2.830650  186.074838    7.606315

(Intercept)    train_x1    train_x2    train_x3 
   2.812936  155.938477    7.133955   10.369536

(Intercept)    train_x1    train_x2    train_x3    train_x4 
   2.842229   67.393780    7.215337   10.247729   22.175315

(Intercept)    train_x1    train_x2    train_x3    train_x4    train_x5 
   2.859045  -17.230461    7.951703   10.554596   21.814597   16.725241

---

## 시도 8

- $y_1=3+x_1+1.5x_2+2x_3+2.5x_4+3x_5+\epsilon$
  - $x_2=x^2_1+\epsilon$
  - $x_3=x^3_1+\epsilon$
  - $x_4=x^4_1+\epsilon$
  - $x_5=x^5_1+\epsilon$
- 1000번 반복
- 1000개 데이터
- 설명변수를 하나씩 제거해나가며 값 확인

In [26]:
result1 = c()
result2 = c()
result2_2 = c()
result2_3 = c()
result2_4 = c()
for (i in 1:1000){
    x1 = runif(1000)
    x2 = x1^2 + rnorm(1000,0,0.01)
    x3 = x1^3 + rnorm(1000,0,0.01)
    x4 = x1^4 + rnorm(1000,0,0.01)
    x5 = x1^5 + rnorm(1000,0,0.01)
    y1= 3 + x1 + 1.5*x2 + 2*x3 + 2.5*x4 + 3*x5 + rnorm(1000)
    train_x1 = x1[1:500]
    train_x2 = x2[1:500]
    train_x3 = x3[1:500]
    train_x4 = x4[1:500]
    train_x5 = x5[1:500]
    train_y1 = y1[1:500]
    test_x1 = x1[501:1000]
    test_x2 = x2[501:1000]
    test_x3 = x3[501:1000]
    test_x4 = x4[501:1000]
    test_x5 = x5[501:1000]
    test_y1 = y1[501:1000]
    M1 = lm(train_y1~train_x1)
    M2 = lm(train_y1~train_x1+train_x2)
    M2_2 = lm(train_y1~train_x1+train_x2+train_x3)
    M2_3 = lm(train_y1~train_x1+train_x2+train_x3+train_x4)
    M2_4 = lm(train_y1~train_x1+train_x2+train_x3+train_x4+train_x5)
    result1[i]=mean((test_y1-predict(M1,data.frame(test_x1)))^2)
    result2[i]=mean((test_y1-predict(M2,data.frame(test_x1,test_x2)))^2)
    result2_2[i]=mean((test_y1-predict(M2_2,data.frame(test_x1,test_x2,test_x3)))^2)
    result2_3[i]=mean((test_y1-predict(M2_3,data.frame(test_x1,test_x2,test_x3,test_x4)))^2)
    result2_4[i]=mean((test_y1-predict(M2_4,data.frame(test_x1,test_x2,test_x3,test_x4,test_x5)))^2)  
    }
print(mean(result1));print(mean(result2));print(mean(result2_2));print(mean(result2_3));print(mean(result2_4))

[1] 14.08265
[1] 15.17156
[1] 15.25551
[1] 15.27434
[1] 15.27908


In [27]:
M2$coefficients;M2_2$coefficients;M2_3$coefficients;M2_4$coefficients

(Intercept)    train_x1    train_x2 
   3.524129   -4.708915   12.877889

(Intercept)    train_x1    train_x2    train_x3 
   2.983878    1.974258   -4.239621   11.629665

(Intercept)    train_x1    train_x2    train_x3    train_x4 
   2.828366    2.619581    0.163239   -3.375276   10.516150

(Intercept)    train_x1    train_x2    train_x3    train_x4    train_x5 
   2.818175    2.433067    1.330597   -3.699660    7.104684    2.866718

---

다중공선성을 어느정도 제거한 모델과 제거하지 않은 모델들을 비교해보았다.
- 설명변수끼리 배수 관계에 있던 모델들과 다르게 거듭제곱 관계에 있던 모델들은 결과가 비슷하긴 하지만 설명변수가 배수관계에 있던 모델만큼 비슷하진 않았다.
- 설명변수끼리 관계가 있고, 또 여러개 존재했을때 확실히 설명변수가 추가될수록 모형의 평균제곱오차가 커지는 경향을 보였다.